In [ ]:
### 주택가격 예측 데이터 불러들이기
# 데이터프레임 변수명 : data
import pandas as pd

data = pd.read_csv("./data/01_회귀_주택가격예측_데이터셋_Row_1000.csv")

data.info()
data.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   면적          1000 non-null   float64
 1   층수          1000 non-null   int64  
 2   역까지_도보시간    1000 non-null   float64
 3   주변_편의시설_점수  1000 non-null   float64
 4   주택_가격       1000 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 39.2 KB


,면적,층수,역까지_도보시간,주변_편의시설_점수,주택_가격
0,121.146284,13,35.025575,9.100563,16601.16745


In [ ]:
### 독립변수와 종속변수로 분리하기
# - 독립변수명 : X (대문자)
# - 종속변수명 : y
X = data.iloc[:, :-1]
y = data["주택_가격"]
print(X.shape, y.shape)


# - 데이터 스케일링 : 스텐다드스케일러 사용
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
print(X_scaled.shape)

# - 훈련 : 테스트 = 8 : 2로 분리
#   -> 변수명 : X_train, y_train, X_test, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1000, 4) (1000,)
(1000, 4)
(800, 4) (800,)
(200, 4) (200,)


### 심층신경망(DNN) 모델로 훈련시키기

- 심층신경망(DNN)으로 훈련 가능한 데이터 형태
    - 이미지 데이터, 일반 수치형 데이터
    - 회귀, 분류분석 모두 가능함

In [ ]:
### 텐서플로우 프레임워크(패키지라고도 칭함)
import tensorflow as tf

### 케라스 라이브러리 불러들이기
from tensorflow import keras

### 시각화 라이브러리 불러들이기
import matplotlib.pyplot as plt

### 넘파이
import numpy as np

### 훈련 및 테스트 데이터로 분류하는 라이브러리
from sklearn.model_selection import train_test_split

### 딥러닝 랜덤 규칙 정의하기
# - 딥러닝에서의 랜덤 규칙은 항상 일정하게 유지되지는 않음
# - 딥러닝 모델 내부에서 훈련을 위한 데이터를 임의로 추출하여 사용(사람이 관여 안함)
tf.keras.utils.set_random_seed(42)
# tf.config.experimental.enable_op_determinism()

- 모델 생성하기
    - 모델명 : model
    - 입력계층 생성 : 출력갯수 64, 활성화함수 relu
    - 은닉계층 생성 : 출력갯수 32, 활성화함수 relu
    - 출력계층 생성 : 출력갯수 ? , 활성화함수 ?
    - 모델 구조 확인까지만(모델 설정전 까지만)

In [ ]:
### 모델 생성
model = keras.Sequential()

### 입력계층
model.add(
    keras.layers.Dense(
        units=64, activation="relu",
        input_shape=(X_train.shape[1], )
    )
)

### 은닉계층
model.add(
    keras.layers.Dense(
        units=32, activation="relu"
    )
)

### 출력계층
model.add(
    keras.layers.Dense(
        units=1,

        # 회귀분석 : 추정값 결정
        activation="linear"
    )
)

### 모델 구조 확인하기
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
### 모델 환경 설정
# - 옵티마이저 : adam 사용
# ** 회귀분석에서는 metrics 정확도는 출력에 사용하지 않음(손실만 사용함)
#    --> metrics 정확도는 "분류분석에서는 사용"
model.compile(
    optimizer="adam",
    ### 회귀분석 시에는 주로 MSE를 이용하여 손실을 측정함
    loss="mean_squared_error"
)

In [ ]:
### 콜백 함수 적용하기
# - 저장할 모델명 : best_linear_model.h5
# - 자동 저장 및 자동 종료 콜백함수 정의하기

### 모델 저장시키는 콜백함수 정의하기
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    # 저장할 위치 지정
    "./model/best_linear_model.h5",

    # epoch 마다 훈련 모델 자동 저장 여부 설정
    #  - save_best_only = True : 모델 성능이 좋아질 때만 자동 저장
    #  - save_best_only = False : epoch 마다 자동 저장
    save_best_only = True
)

### 훈련 자동 종료 시키는 콜백함수 정의
early_stopping_cb = keras.callbacks.EarlyStopping(
    # 더 이상 좋아지지 않는 epoch의 갯수 지정
    #  - 가장 좋은 시점의 epoch 이후 2번 더 수행 후
    #    -- 그래도 좋와지지 않으면 종료시킨다는 의미
    #  - 주로 2 또는 3을 사용
    patience = 2,

    # 종료 시점에 가장 낮은 검증 손실일 때의 하이퍼파라메터로 모델 업데이트 시킴
    #  - 가중치 업데이트
    restore_best_weights = True
)

checkpoint_cb, early_stopping_cb

(<keras.callbacks.ModelCheckpoint at 0x1e0487bf820>,
 <keras.callbacks.EarlyStopping at 0x1e0487bf310>)

In [ ]:
### 훈련 시키기
# - 검증 데이터로는 테스트 데이터 사용

### 훈련 시 콜백함수 사용하기
history = model.fit(
    X_train, y_train, epochs=1000,
    validation_data=(X_test, y_test),

    # 콜백함수 정의하기
    callbacks = [checkpoint_cb, early_stopping_cb]
)

Epoch 1/1000
25/25 [==============================] - 2s 12ms/step - loss: 455115968.0000 - val_loss: 431334368.0000
Epoch 2/1000
25/25 [==============================] - 0s 5ms/step - loss: 455063264.0000 - val_loss: 431275200.0000
Epoch 3/1000
25/25 [==============================] - 0s 5ms/step - loss: 454980224.0000 - val_loss: 431172128.0000
Epoch 4/1000
25/25 [==============================] - 0s 8ms/step - loss: 454837504.0000 - val_loss: 430991840.0000
Epoch 5/1000
25/25 [==============================] - 0s 5ms/step - loss: 454593696.0000 - val_loss: 430701824.0000
Epoch 6/1000
25/25 [==============================] - 0s 5ms/step - loss: 454209344.0000 - val_loss: 430253760.0000
Epoch 7/1000
25/25 [==============================] - 0s 6ms/step - loss: 453635456.0000 - val_loss: 429605856.0000
Epoch 8/1000
25/25 [==============================] - 0s 6ms/step - loss: 452831072.0000 - val_loss: 428706400.0000
Epoch 9/1000
25/25 [==============================] - 0s 7ms/step - los

In [ ]:
### 최적의 모델 불러들이기
# - 훈련 성능 평가
# - 검증 성능 평가

model_all = keras.models.load_model("./model/best_linear_model.h5")
train_score = model_all.evaluate(X_train, y_train)
val_score = model_all.evaluate(X_test, y_test)

print(f"훈련 : {train_score}")
print(f"검증 : {val_score}")

7/7 [==============================] - 0s 4ms/step - loss: 439843.0000
훈련 : 375378.09375
검증 : 439843.0


In [ ]:
### 테스트 데이터의 0번째 값으로 예측하기
test_pred = model_all.predict(X_test[:1])
test_pred, y_test[:1]

(array([[37236.58]], dtype=float32),
 521    37736.352245
 Name: 주택_가격, dtype: float64)

###